<a href="https://colab.research.google.com/github/fabrizioca2/ACS2024-1_GRUPO_06/blob/Aldair-Solis/algoritmo_geneticopreg3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install deap
!pip install control
import numpy as np
import control as ctrl
from deap import base, creator, tools, algorithms

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.6/549.6 kB 8.0 MB/s eta 0:00:00


In [ ]:
# Sistema de ejemplo: Planta G(s) = 1 / (s^2 + 3s + 2)
# Parámetros del sistema
M = 0.8  # Masa del carro (kg)
m = 0.23  # Masa del péndulo (kg)
l = 0.3  # Longitud del péndulo (m)
g = 9.81 # Aceleración de la gravedad (m/s²)

In [ ]:
#Se define numerador y denominador
numerador = [1]
denominador = [M*l, 0, -(M+m)*g]

In [ ]:
planta = ctrl.TransferFunction(numerador, denominador)

In [ ]:
# Función objetivo: Evaluar rendimiento del controlador PID
def evaluar_pid(individuo):
    Kp, Ki, Kd = individuo
    # Controlador PID
    controlador = ctrl.TransferFunction([Kd, Kp, Ki], [1, 0])
    # Lazo cerrado
    sistema_lazo_cerrado = ctrl.feedback(controlador * planta)
    # Simulación de la respuesta al escalón
    t, yout = ctrl.step_response(sistema_lazo_cerrado)
    # Error: Deseamos que la salida siga un escalón unitario
    error = 1 - yout
    # ISE: Integral del error al cuadrado
    ise = np.sum(error**2) * (t[1] - t[0])
    return ise,

In [ ]:
# Configuración del algoritmo genético
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimizar ISE
creator.create("Individual", list, fitness=creator.FitnessMin)
toolbox = base.Toolbox()

In [ ]:
# Definir individuos (Kp, Ki, Kd)
toolbox.register("attr_float", np.random.uniform, 0, 10)  # Rango de [0, 10]
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, 3)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [ ]:
# Operadores genéticos
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluar_pid)

In [ ]:
# Proceso evolutivo
population = toolbox.population(n=50)
ngen = 20  # Número de generaciones
cxpb = 0.7  # Probabilidad de cruzamiento
mutpb = 0.2  # Probabilidad de mutación

In [ ]:
# Se ejecuta algoritmo genético
resultados = algorithms.eaSimple(population, toolbox, cxpb, mutpb, ngen, verbose=True)
# Obtener los mejores parámetros PID
mejor_individuo = tools.selBest(population, k=1)[0]
print("Mejores ganancias PID encontradas:")
print(f"Kp = {mejor_individuo[0]}, Ki = {mejor_individuo[1]}, Kd = {mejor_individuo[2]}")

/usr/local/lib/python3.10/dist-packages/control/timeresp.py:1197: RuntimeWarning: overflow encountered in matmul
  xout[:, i] = (Ad @ xout[:, i-1]
/usr/local/lib/python3.10/dist-packages/control/timeresp.py:1197: RuntimeWarning: invalid value encountered in matmul
  xout[:, i] = (Ad @ xout[:, i-1]
/usr/local/lib/python3.10/dist-packages/control/timeresp.py:1199: RuntimeWarning: overflow encountered in matmul
  yout = C @ xout + D @ U
<ipython-input-5-4d6c465d2b42>:13: RuntimeWarning: overflow encountered in square
  ise = np.sum(error**2) * (t[1] - t[0])


gen	nevals
0  	50    
1  	37    
2  	34    
3  	36    
4  	37    
5  	40    
6  	42    
7  	47    
8  	38    
9  	35    
10 	40    
11 	37    
12 	38    
13 	37    
14 	34    
15 	43    
16 	37    
17 	36    
18 	35    
19 	40    
20 	40    
Mejores ganancias PID encontradas:
Kp = 24.0077740907898, Ki = 9.346448746952095, Kd = 5.9176208226312905
